# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to build your neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

We want to be able to train our model on an [accelerator](https://pytorch.org/docs/stable/torch.html#accelerators) such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

In [2]:
try:
    device = torch.accelerator.current_accelerator.type if torch.accelerator.is_available() else "cpu"
    print(f"Using {device} device")
except:
    device = 'cpu'
    print('The CPU-only version of PyTorch is installed -- no torch.accelerator module available.')

The CPU-only version of PyTorch is installed -- no torch.accelerator module available.


## Define the Class

We define our neural network by subclassing `nn.Module` and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [3]:
class NeuralNetwork(nn.Module):
    '''
    This is a 2-hidden-layer multilayer perceptron (MLP) with 512 units per
    hidden layer and ReLU activations. It maps a 28x28 grayscale image to a
    10-class classification output.
    Forward pass flow:
        [N,1,28,28] -> [N,784] -> [N,512] -> [N,512] -> [N,10]
    '''
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(  # ordered container, runs layers in sequence.
            nn.Linear(28*28, 512),  # Layer 1: fully connected, projects flattened image -> 512 features
            nn.ReLU(),              # Activation 1: applies ReLU element-wise (max(0,x)), introducing nonlinearities
            nn.Linear(512, 512),    # Layer 2: hidden layer, keeps feature dimension 512
            nn.ReLU(),              # Activation 2: applies ReLU element-wise (max(0,x)), introducing nonlinearities
            nn.Linear(512, 10)      # Layer 3: output layer, 10 units = 1 unit per FashionMNIST class
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's `forward`, along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of eah output. We get the prediction probabilities by passing it through an instance of `nn.Softmax` module.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


## Model Layers

Let's break down the layers in the fashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### `nn.Flatten`

We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### `nn.Linear`

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### `nn.ReLU`

Non-linear activations are what create the complex mappings between the model's inputs and outputs. They are applies after linear transformations to introduce *nonlinearity*, helping neural networks to approximate more complex functions.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there's other activations to introduce non-linearity in your model.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.4801, -0.3691,  0.4417, -0.0261,  0.4218,  0.0454, -0.0938, -0.0899,
          0.6204,  0.3612,  0.2228,  0.9825, -0.3221, -0.2441,  0.5065,  0.0515,
         -0.0164,  0.3303, -0.1608, -0.2209],
        [ 0.3995, -0.2207,  0.1956, -0.0415,  0.0428, -0.0914, -0.3814, -0.1659,
          0.3649,  0.1680,  0.3953,  0.4469, -0.4674, -0.3793,  0.1014,  0.6099,
          0.2419, -0.1294, -0.1382, -0.3718],
        [ 0.4446, -0.1656,  0.6187,  0.2144,  0.3120, -0.3731, -0.1368, -0.2520,
          0.6985,  0.2370,  0.4992,  0.4284, -0.1564, -0.1241,  0.3514,  0.4877,
          0.0074,  0.1786, -0.4963, -0.7153]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.4801, 0.0000, 0.4417, 0.0000, 0.4218, 0.0454, 0.0000, 0.0000, 0.6204,
         0.3612, 0.2228, 0.9825, 0.0000, 0.0000, 0.5065, 0.0515, 0.0000, 0.3303,
         0.0000, 0.0000],
        [0.3995, 0.0000, 0.1956, 0.0000, 0.0428, 0.0000, 0.0000, 0.0000, 0.3649,
         0.1680, 0.3953, 0.4469, 0.0000, 0.0000, 0.10

### `nn.Sequential`

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### `nn.Softmax`

The last linear layer of the neural network returns *logits* - raw values in \[-infty, infty\] - which are passed to the [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values \[0, 1\] representing the model's predicted probabilities for each class. `dim` parameter indicated the dimension along which the values must sum to 1.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters

Many layers inside a neural network are *parameterized*, i.e., have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0340, -0.0118, -0.0042,  ...,  0.0048, -0.0260, -0.0048],
        [-0.0175,  0.0044, -0.0259,  ...,  0.0286,  0.0203, -0.0347]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0322,  0.0206], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0127, -0.0041, -0.0177,  ..., -0.0060, -0.0110,  0.0129],
        [ 0.0323,  0.0264,  0.0414,  ...,  0.0208, -0.0281, -0.0021]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 